In [23]:
#  DAY 04 - TASK 2 

import pandas as pd
import requests
import pymysql
from bs4 import BeautifulSoup

# Establishing Connection with Database [MYSQL] 
connection = pymysql.connect(host="127.0.0.1", port=3306, user="root", passwd="ramzan", database="hadiths")
cursor = connection.cursor()

#Scrapping from Website ONE (https://40hadithnawawi.com/)
url= 'https://40hadithnawawi.com/'
plain_HTML= requests.get(url)                          #Loading html's plain text from URL
soup = BeautifulSoup(plain_HTML.text, "html.parser")   #Parsing the data
# print(soup.prettify())


In [24]:
subHeadings = soup.findAll('p'); #Getting All the subheadings of hadiths

import re #Removing HTML Tags & splitting hadiths names:
cleanER=re.compile('<.*?>')
subHeadings=str(subHeadings)
subHeadings=subHeadings.replace("[",'')
subHeadings=subHeadings.replace("]",'')
subHeadings=subHeadings.replace(",",'')
subHeadings=subHeadings.split("Hadith")

description=[]

# #Accessing SUB URLS of Hadiths Pages:
subURL=str(soup.findAll('a',class_='flex w-full group text-gray-800')) #Locate SubURL
subURL=subURL.split('href=\"')
allSubURLs=[]
for i in range (len(subURL)):
    if(i>0):
        subURLCLEANED=subURL[i].split("\"><")
        subURLCLEANED=subURLCLEANED[0]
        allSubURLs.append(url+subURLCLEANED)
#ALL SUB URLS OBTAINED TILL HERE.

#NOW ACCESSING SUBURLS AND OBTAINING REQUIRED DATA:

for j in range (len(allSubURLs)):
    plain_HTML2= requests.get(allSubURLs[j])                          #Loading html's plain text from URL
    soup2 = BeautifulSoup(plain_HTML2.text, "html.parser")   #Parsing the data
    desc=soup2.findAll('p')
    text=''
    for r in range (len(desc)):
        if (r>0):
            test=str(desc[r])
            if(test.find("class=\"leading-relaxed\"")>=0): #identifies where description of hadiths stops
                break
            textTEMP=str(desc[r].text.strip())
            text=text+' '+textTEMP
            
    description.append(text)
print(description)

[' It is narrated on the authority of Amir al-Mu\'minin (Leader of the Believers), Abu Hafs \'Umar bin al-Khattab (may Allah be pleased with him), who said: I heard the Messenger of Allah (peace be upon him), say "Actions are according to intentions, and everyone will get what was intended. Whoever migrates with an intention for Allah and His messenger, the migration will be for the sake of Allah and his Messenger. And whoever migrates for worldly gain or to marry a woman, then his migration will be for the sake of whatever he migrated for." Related by Bukhari & Muslim عَنْ أَمِيرِ الْمُؤْمِنِينَ أَبِي حَفْصٍ عُمَرَ بْنِ الْخَطَّابِ رَضِيَ اللهُ عَنْهُ قَالَ: سَمِعْتُ رَسُولَ اللَّهِ صلى الله عليه وسلم يَقُولُ: " إنَّمَا الْأَعْمَالُ بِالنِّيَّاتِ، وَإِنَّمَا لِكُلِّ امْرِئٍ مَا نَوَى، فَمَنْ كَانَتْ هِجْرَتُهُ إلَى اللَّهِ وَرَسُولِهِ فَهِجْرَتُهُ إلَى اللَّهِ وَرَسُولِهِ، وَمَنْ كَانَتْ هِجْرَتُهُ لِدُنْيَا يُصِيبُهَا أَوْ امْرَأَةٍ يَنْكِحُهَا فَهِجْرَتُهُ إلَى مَا هَاجَرَ إلَيْهِ".

In [25]:
#Scratching from Website TWO (https://sunnah.com/):
urlWEB2= 'https://sunnah.com/'
web2_HTML= requests.get(urlWEB2)                          #Loading html's plain text from URL
soupWeb2 = BeautifulSoup(web2_HTML.text, "html.parser")   #Parsing the data
# print(soupWeb2.prettify())

In [26]:
#IN THIS WEBSITE THERE ARE TWO LEVELS OF SUB URLS

#GETTING FIRST LEVEL SUB URLS (total 17):

subURL_level1= str(soupWeb2.findAll('div',class_='collection_title'))
subURL_level1=subURL_level1.split("<a href=\"")
subURL_level1_ALL=[]

for i in range (len(subURL_level1)):
    if(i>0):
        sub2=subURL_level1[i].split("\" style") #Cleaning SUB URLS of LEVEL 1
        subURL_level1_ALL.append(str(urlWEB2)+sub2[0])

# --------------------------------> TILL HERE LEVEL 1 URLS RETREIVED 

#NOW ACCESSING LEVEL 2 SUB URLS:-

titles_level2_ALL=[]
subURL_level2_ALL=[]
titleCount=0
co=43

for j in range (1):
   
      plain_level1_HTML= requests.get(subURL_level1_ALL[j])                          
      soupl1 = BeautifulSoup(plain_level1_HTML.text, "html.parser")   
      level2_title=soupl1.findAll('div',class_="english english_book_name")
      
    
      for k in range (len(level2_title)): #CLEANING THE LEVEL 2 TITLE
          titles_level2_ALL.append(str(level2_title[k].text.strip())) #FINDING SUB URL in LEVEL 2 Corresponding to the title
          subURL_level2=str(soupl1.findAll('div',class_="book_title title"))
          subURL_level2=subURL_level2.split("<a href=\"")
          for i in range (len(subURL_level2)):
              if(i>0):
                  cleanedSubURL=subURL_level2[i].split("\" style")
                  tempSUB=cleanedSubURL[0].split("\">")
                  subURL_level2_ALL.append(str(urlWEB2)+tempSUB[0])
#       ---------------> Till here level 2 sub urls against the title of level 1 has been retreived
#Now Accessing Level 2 to finally retreive required information

      for s in range(1):
           plain_level2_HTML=requests.get(subURL_level2_ALL[s])
           soupl2=BeautifulSoup(plain_level2_HTML.text,"html.parser")
           descriptionAtLast=soupl2.findAll('div',class_='text_details')
           tempDESC="" #Cleaning description:
           for c in range (len(descriptionAtLast)):
              tempDESC=tempDESC+" "+str(descriptionAtLast[c].text.strip())
      description.append(tempDESC)
      subHeadings.append(str(co)+' '+titles_level2_ALL[titleCount])
      titleCount+=titleCount
      co+=co
          
print(description)
print(subHeadings)


[' It is narrated on the authority of Amir al-Mu\'minin (Leader of the Believers), Abu Hafs \'Umar bin al-Khattab (may Allah be pleased with him), who said: I heard the Messenger of Allah (peace be upon him), say "Actions are according to intentions, and everyone will get what was intended. Whoever migrates with an intention for Allah and His messenger, the migration will be for the sake of Allah and his Messenger. And whoever migrates for worldly gain or to marry a woman, then his migration will be for the sake of whatever he migrated for." Related by Bukhari & Muslim عَنْ أَمِيرِ الْمُؤْمِنِينَ أَبِي حَفْصٍ عُمَرَ بْنِ الْخَطَّابِ رَضِيَ اللهُ عَنْهُ قَالَ: سَمِعْتُ رَسُولَ اللَّهِ صلى الله عليه وسلم يَقُولُ: " إنَّمَا الْأَعْمَالُ بِالنِّيَّاتِ، وَإِنَّمَا لِكُلِّ امْرِئٍ مَا نَوَى، فَمَنْ كَانَتْ هِجْرَتُهُ إلَى اللَّهِ وَرَسُولِهِ فَهِجْرَتُهُ إلَى اللَّهِ وَرَسُولِهِ، وَمَنْ كَانَتْ هِجْرَتُهُ لِدُنْيَا يُصِيبُهَا أَوْ امْرَأَةٍ يَنْكِحُهَا فَهِجْرَتُهُ إلَى مَا هَاجَرَ إلَيْهِ".

In [27]:
#Inserting ALL Data Scrapped into Database [mySQL]:

for i in range (len(subHeadings)):
    subHeadings[i]=re.sub(cleanER,'',subHeadings[i])
    # insert into database
    if(i>0):
        cursor.execute("INSERT INTO hadiths (name, description) VALUES (%s, %s)", (subHeadings[i], description[i-1]))

connection.commit()
connection.close()
print("Data inserted successfully in the database.")


Data inserted successfully in the database.
